## diabetes 예제 딥러닝 회귀
outcome 삭제 후 bmi 회귀

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [19]:
file_path = './diabetes.csv'
data = pd.read_csv(file_path)

data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [20]:
# 데이터 확인
print("데이터 형태:", data.shape)
print("\n컬럼 정보:\n", data.columns)
print("\n클래스 분포:\n", data["BMI"].value_counts())
print("\BMI 분포:\n", data["BMI"].describe())

데이터 형태: (768, 9)

컬럼 정보:
 Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

클래스 분포:
 BMI
32.0    13
31.6    12
31.2    12
0.0     11
32.4    10
        ..
36.7     1
41.8     1
42.6     1
42.8     1
46.3     1
Name: count, Length: 248, dtype: int64
\BMI 분포:
 count    768.000000
mean      31.992578
std        7.884160
min        0.000000
25%       27.300000
50%       32.000000
75%       36.600000
max       67.100000
Name: BMI, dtype: float64


In [21]:
# 결측치 확인
print("=== 결측치 현황 ===")
print(data.isnull().sum())

=== 결측치 현황 ===
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [22]:
# 불필요한 칼럼 제거
data = data.drop(['Outcome'], axis=1) # axis=1: 열(columns)을 기준으로 작업
print(data.columns)

# 특성(X)과 레이블(y) 분리
X = data.drop("BMI", axis=1)
y = data["BMI"]

# 훈련/테스트 세트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


In [23]:
# 데이터셋 Shape 확인
print("\n=== 데이터셋 Shape ===")
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)



=== 데이터셋 Shape ===
X_train_scaled shape: (614, 7)
X_test_scaled shape: (154, 7)
y_train shape: (614,)
y_test shape: (154,)


In [24]:
models = {
    "SVM": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Linear Regression": LinearRegression(),
    "KNN": KNeighborsRegressor()
}

print("=== 머신러닝 모델 성능 ===")
for name, model in models.items():
    model.fit(X_train_scaled, y_train) # 모델 학습
    y_pred = model.predict(X_test_scaled) # 테스트 데이터 예측

    # 성능지표 계산
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
 
    # 결과 출력
    print(f"\n======={name}=======")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.4f}")


=== 머신러닝 모델 성능 ===

=======SVM=======
RMSE: 7.27
MAE: 4.95
R²: 0.2565

=======Decision Tree=======
RMSE: 10.76
MAE: 7.42
R²: -0.6296

=======Random Forest=======
RMSE: 7.01
MAE: 5.28
R²: 0.3088

=======Linear Regression=======
RMSE: 7.23
MAE: 5.21
R²: 0.2651

=======KNN=======
RMSE: 7.21
MAE: 5.13
R²: 0.2685


### 텐서플로우 회귀 모델

In [26]:
# 모델 구조 정의 (배치 정규화 및 드롭아웃 추가)
# model = Sequential([
#     Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
#     BatchNormalization(),  # 배치 정규화 추가
#     Dropout(0.3),          # 드롭아웃 추가
#     Dense(32, activation='relu'),
#     Dense(1)  # 출력층
# ])

# 모델 구조 정의 (배치 정규화 및 드롭아웃 추가)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),  # 배치 정규화 추가
    Dropout(0.3),          # 드롭아웃 추가
    Dense(32, activation='relu'),
    Dense(1)  # 출력층
])


optimizer = Adam(learning_rate=0.001) # 학습률

# 모델 컴파일 (회귀 설정)
model.compile(
    optimizer=optimizer,
    loss='mse',  # 평균 제곱 오차
    metrics=['mae']  # 평균 절대 오차
)

# 조기 종료 콜백 추가
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2, #X_train_scaled의 20%를 검증 데이터로 분리
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],  # 조기 종료 적용  
    verbose=1
)

# 테스트 평가
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
y_pred_dl = model.predict(X_test_scaled).flatten()

# 성능 지표 계산
rmse_dl = np.sqrt(mean_squared_error(y_test, y_pred_dl))
mae_dl = mean_absolute_error(y_test, y_pred_dl)
r2_dl = r2_score(y_test, y_pred_dl)

print("\n=== 딥러닝 모델 (TensorFlow) ===")
print(f"RMSE: {rmse_dl:.2f}")
print(f"MAE: {mae_dl:.2f}")
print(f"R²: {r2_dl:.4f}")

Epoch 1/50


/Users/kimdoyeon/Projects/SchoolProjects/AI-Study/week4/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1064.0604 - mae: 31.7710 - val_loss: 1083.4910 - val_mae: 32.1743
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1041.6411 - mae: 31.4435 - val_loss: 1063.2708 - val_mae: 31.8698
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1050.7955 - mae: 31.4747 - val_loss: 1042.8342 - val_mae: 31.5582
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 998.5560 - mae: 30.7779 - val_loss: 1019.4130 - val_mae: 31.2005
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 965.9828 - mae: 30.2254 - val_loss: 992.0033 - val_mae: 30.7699
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 963.2502 - mae: 30.0299 - val_loss: 960.5149 - val_mae: 30.2562
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 938.8483 - mae: 29.7587 - val_loss: 921.9589 - val_mae: 29.6197
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 879.9637 - mae: 28.7167 - val_loss: 878.1982 - val_mae: 28.8757
Epoch 9/50
16/16 ━━━━━━━━━━

### PyTorch 회귀 모델

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

#  1. 데이터 준비 및 전처리
# 스케일링된 데이터를 PyTorch 텐서로 변환
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)  
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)    
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# 검증 세트 분리 (스케일링된 데이터 사용)
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train_scaled, y_train, test_size=0.2, random_state=42
)

# 검증 데이터 텐서 변환
X_train_sub_tensor = torch.tensor(X_train_sub, dtype=torch.float32)
y_train_sub_tensor = torch.tensor(y_train_sub.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)


# 2. 데이터 로더 설정
# Dataset 생성 (특성-레이블 쌍)
train_dataset = TensorDataset(X_train_sub_tensor, y_train_sub_tensor) #훈련
val_dataset = TensorDataset(X_val_tensor, y_val_tensor) #검증
test_dataset = TensorDataset(X_test_tensor, y_test_tensor) #테스트

# 데이터 로더생성 (배치처리 + 셔플링)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) #훈련 # 무작위
val_loader = DataLoader(val_dataset, batch_size=32) # 검증
test_loader = DataLoader(test_dataset, batch_size=32) #테스트

# 3. 신경망 모델 정의 (회귀)
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        # 신경망 구조 정의
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64), # 입력층 => 은닉층 1 (64개 뉴런)
            nn.ReLU(), # ReLU 활성화 함수
            nn.Linear(64, 32), # 은닉층 1 => 은닉층 2 (32개 뉴런)
            nn.ReLU(),
            nn.Linear(32, 1) # 은닉층 2 => 출력층 (1개 뉴런)
        )

    def forward(self, x):
        # 순전파: 입력 데이터를 신경망에 통과 시킴
        return self.model(x)

# 4. 학습 설정
# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 인스턴스 생성, 지정된 장치 이동
model = RegressionModel(input_dim=X_train_scaled.shape[1]).to(device)

# 손실 함수: 평균 제곱 오차(MSE)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # 옵티마이저

# 5. 학습 루프 (조기종료 포함)
num_epochs = 50
best_val_loss = float('inf') # 최적 검증 손실 초기값
patience = 10 # 개선 없는 에포크 허용 횟수
patience_counter = 0 # 현재 허용 횟수 카운터

for epoch in range(50):
    # 훈련 단계
    model.train() # 학습모드
    total_loss = 0.0
    for X_batch, y_batch in train_loader:
        # 데이터를 저장된 장치로 이동
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad() # 기울기 초기화
        outputs = model(X_batch) # 순전파: 예측값 계산
        loss = criterion(outputs, y_batch) # 손실계산
        loss.backward() # 역전파: 기울기 계산
        optimizer.step() # 가중치 업데이트
        
        total_loss += loss.item() # 누적 손실 업데이트

    # 검증 단계 
    model.eval() # 평가 모드
    val_loss = 0.0
    with torch.no_grad(): # 기울기 계산 비활
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item()
    
    val_loss /= len(val_loader) # 검증 손실 평균 계산
    print(f"Epoch {epoch+1} | Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f}")
    
    # 조기 종료 로직
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early Stopping!")
            break

# 평가
model.eval() #평가모드
preds, actuals = [], []

# 기울기 계산 비활
with torch.no_grad():
    for X_batch, y_batch in test_loader:

        # 테스트 데이터를 장치로 이동
        X_batch = X_batch.to(device)

        # 예측 수행 및 CPU로 이동, numpy배열로 변환
        outputs = model(X_batch).cpu().numpy().flatten()

        #결과 저장
        preds.extend(outputs)
        actuals.extend(y_batch.numpy().flatten())

# 성능 지표 계산
rmse = np.sqrt(mean_squared_error(actuals, preds))
mae = mean_absolute_error(actuals, preds)
r2 = r2_score(actuals, preds)

print("\n=== 딥러닝 모델 성능(PyTorch) ===")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.4f}")

Epoch 1 | Train Loss: 1075.4898 | Val Loss: 981.5896
Epoch 2 | Train Loss: 1044.5289 | Val Loss: 941.2436
Epoch 3 | Train Loss: 983.8487 | Val Loss: 878.1596
Epoch 4 | Train Loss: 909.0608 | Val Loss: 787.6068
Epoch 5 | Train Loss: 802.8614 | Val Loss: 664.7269
Epoch 6 | Train Loss: 652.2064 | Val Loss: 515.6853
Epoch 7 | Train Loss: 487.1587 | Val Loss: 362.6289
Epoch 8 | Train Loss: 330.3936 | Val Loss: 230.4172
Epoch 9 | Train Loss: 201.4542 | Val Loss: 148.1116
Epoch 10 | Train Loss: 126.5942 | Val Loss: 111.4011
Epoch 11 | Train Loss: 94.1970 | Val Loss: 102.2721
Epoch 12 | Train Loss: 82.2293 | Val Loss: 99.0965
Epoch 13 | Train Loss: 75.8187 | Val Loss: 94.6403
Epoch 14 | Train Loss: 71.6526 | Val Loss: 91.8718
Epoch 15 | Train Loss: 67.7851 | Val Loss: 89.2898
Epoch 16 | Train Loss: 65.3370 | Val Loss: 88.2689
Epoch 17 | Train Loss: 62.7833 | Val Loss: 85.9697
Epoch 18 | Train Loss: 60.7464 | Val Loss: 85.1897
Epoch 19 | Train Loss: 60.7541 | Val Loss: 83.8688
Epoch 20 | Train 